# Tough Shell Script Assignment: Log File Rotation and Archiving Automation

This assignment will challenge you to write a robust shell script to automate the process of rotating and archiving log files. This is a common and crucial task in system administration to prevent log files from consuming excessive disk space and to retain historical data for auditing or debugging purposes. While you cannot run the full automation directly *within* this Jupyter Notebook (as it involves background processes and system scheduling), this notebook will guide you through building the script, understanding its components, and setting it up for real-world use.

## **Important Notes & Setup Instructions (READ FIRST!):**

1.  **Environment:** This assignment assumes a Unix-like operating system (Linux, macOS, WSL on Windows) where bash shell scripts can be executed.
2.  **File Creation:** You will be instructed to copy the shell script code into a `.sh` file and make it executable outside this notebook.
3.  **Running the Script:** You'll execute the script from your terminal.
4.  **Crontab (for Automation):** The final step for automation involves setting up a cron job, which is done directly on your system's terminal, not within Jupyter.
5.  **Simulating Logs:** We will use Python code within this notebook to simulate the creation of log files for testing your script.


## Part 1: Understanding Log Rotation and Archiving Concepts (10 points)

Before writing the script, ensure you understand the core concepts.

### 1.1 What is Log Rotation?
Explain in your own words why log rotation is necessary for system health and stability.

**Your Answer:**
*(Write your answer here)*

### 1.2 What is Log Archiving?
Differentiate between log rotation and log archiving. Why would you archive logs instead of just deleting old ones?

**Your Answer:**
*(Write your answer here)*


## Part 2: Simulating Log Files (15 points)

Let's create some dummy log files that your script will operate on.

In [ ]:
import os
from datetime import datetime, timedelta
import random

LOG_DIR = "./app_logs"
APP_LOG_FILE = os.path.join(LOG_DIR, "app.log")
ERROR_LOG_FILE = os.path.join(LOG_DIR, "error.log")

def create_dummy_log_entry(log_type):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    user = random.choice(["user1", "user2", "admin", "guest"])
    messages = {
        "app": [
            "User {} logged in.".format(user),
            "Processed request for {}.".format(random.choice(["/dashboard", "/api/data", "/settings"])),
            "Application heartbeat: OK.",
            "Data record #{} updated.".format(random.randint(1000, 9999)),
        ],
        "error": [
            "[ERROR] Failed to connect to database for user {}.".format(user),
            "[WARNING] High memory usage detected.",
            "[CRITICAL] Unhandled exception in module {}.py".format(random.choice(["auth", "data_processor"])),
            "[ERROR] API endpoint '/api/data' returned 500 for user {}.".format(user),
        ]
    }
    return f"{timestamp} - {random.choice(messages[log_type])}"

def generate_dummy_logs(num_entries_app=50, num_entries_error=10):
    os.makedirs(LOG_DIR, exist_ok=True)

    print(f"Generating {num_entries_app} app log entries to {APP_LOG_FILE}")
    with open(APP_LOG_FILE, "w") as f: # Overwrite for fresh test
        for _ in range(num_entries_app):
            f.write(create_dummy_log_entry("app") + "\n")

    print(f"Generating {num_entries_error} error log entries to {ERROR_LOG_FILE}")
    with open(ERROR_LOG_FILE, "w") as f: # Overwrite for fresh test
        for _ in range(num_entries_error):
            f.write(create_dummy_log_entry("error") + "\n")

    print("Dummy log files created.")

# Run this cell to create your dummy log files
generate_dummy_logs()

# Verify creation
print(f"\nContent of {APP_LOG_FILE} (first 3 lines):")
if os.path.exists(APP_LOG_FILE):
    with open(APP_LOG_FILE, 'r') as f:
        for i, line in enumerate(f):
            print(line.strip())
            if i >= 2: break
else:
    print("File not found.")


## Part 3: Shell Script for Log Rotation and Archiving (50 points)

Now, let's write the core shell script. Create a new file named `rotate_logs.sh` in the same directory as this notebook and paste the following content. **Fill in the missing parts marked with `TODO`.**

### `rotate_logs.sh`

```bash
#!/bin/bash

# --- Configuration Variables ---
LOG_DIR="./app_logs"                      # Directory where log files are located
ARCHIVE_DIR="./archived_logs"             # Directory to store archived logs
LOG_FILES=("app.log" "error.log")         # Array of log file names to rotate
RETENTION_DAYS=7                          # Number of days to keep archived logs
MAX_ROTATIONS=3                           # Number of old log files to keep (e.g., app.log.1, app.log.2)
DATE_FORMAT="+%Y-%m-%d_%H%M%S"            # Timestamp format for archived files

# --- Logging Function ---
log_message() {
    local type="$1"
    local message="$2"
    echo "[$(date +'%Y-%m-%d %H:%M:%S')] [$type] $message"
}

# --- Create necessary directories if they don't exist ---
log_message "INFO" "Ensuring log directories exist..."
mkdir -p "$LOG_DIR" || { log_message "ERROR" "Failed to create log directory: $LOG_DIR"; exit 1; }
mkdir -p "$ARCHIVE_DIR" || { log_message "ERROR" "Failed to create archive directory: $ARCHIVE_DIR"; exit 1; }

log_message "INFO" "Starting log rotation and archiving process."

# --- Rotate Log Files ---
for log_file in "${LOG_FILES[@]}"; do
    FULL_LOG_PATH="$LOG_DIR/$log_file"
    log_message "INFO" "Processing log file: $FULL_LOG_PATH"

    if [[ ! -f "$FULL_LOG_PATH" ]]; then
        log_message "WARNING" "Log file not found, skipping: $FULL_LOG_PATH"
        continue
    fi

    # Step 1: Archive the current log file with a timestamp
    # Example: app.log -> app.log_2025-06-05_134500.gz
    TIMESTAMP=$(date "$DATE_FORMAT")
    ARCHIVED_FILENAME="${log_file}_${TIMESTAMP}.gz"
    FULL_ARCHIVE_PATH="$ARCHIVE_DIR/$ARCHIVED_FILENAME"

    log_message "INFO" "Archiving $log_file to $FULL_ARCHIVE_PATH"
    # TODO: Implement archiving. Use `tar -czf` for compression.
    # Make sure to handle the case where the log file is empty.
    # Hint: Check file size before archiving to avoid empty archives.
    # Use `cat` and `gzip` or `tar` to archive.
    # If using `tar`, remember to change directory or specify full path.
    
    # Your code here for archiving
    # Example (using gzip directly, consider tar if multiple files per archive are needed later):
    if [[ -s "$FULL_LOG_PATH" ]]; then # Check if file exists and is not empty
        gzip -c "$FULL_LOG_PATH" > "$FULL_ARCHIVE_PATH" 2>/dev/null
        if [[ $? -ne 0 ]]; then
            log_message "ERROR" "Failed to archive $log_file. Continuing to next log."
            continue
        fi
    else
        log_message "INFO" "$log_file is empty. Skipping archiving."
    fi

    # Step 2: Truncate the original log file
    log_message "INFO" "Truncating $log_file"
    # TODO: Implement truncation. How do you empty a file while keeping its permissions?
    # Your code here for truncation
    > "$FULL_LOG_PATH" || { log_message "ERROR" "Failed to truncate $log_file"; continue; }

    # Step 3: Delete old rotated files (if they exist) beyond MAX_ROTATIONS
    # This part shifts app.log.2 to app.log.3, app.log.1 to app.log.2, etc.
    # TODO: Implement the rotation of .N files.
    # Loop from MAX_ROTATIONS down to 1.
    # Example: app.log.3 will be deleted, app.log.2 -> app.log.3, app.log.1 -> app.log.2

    for ((i=$MAX_ROTATIONS; i>=1; i--)); do
        OLD_ROTATED_FILE="$FULL_LOG_PATH.$i"
        NEW_ROTATED_FILE="$FULL_LOG_PATH.$((i+1))"
        if [[ -f "$OLD_ROTATED_FILE" ]]; then
            if [[ $i -eq $MAX_ROTATIONS ]]; then
                log_message "INFO" "Deleting old rotated file: $OLD_ROTATED_FILE"
                rm -f "$OLD_ROTATED_FILE" || log_message "WARNING" "Failed to delete $OLD_ROTATED_FILE"
            else
                log_message "INFO" "Renaming $OLD_ROTATED_FILE to $NEW_ROTATED_FILE"
                mv "$OLD_ROTATED_FILE" "$NEW_ROTATED_FILE" || log_message "ERROR" "Failed to rename $OLD_ROTATED_FILE"
            fi
        fi
    done

    # Step 4: Rename current log to .1 (if it has content, or if just truncated)
    # Even if truncated, it should become .1 for consistency
    CURRENT_LOG_TO_ONE="$FULL_LOG_PATH.1"
    log_message "INFO" "Renaming $log_file to $CURRENT_LOG_TO_ONE"
    # TODO: Implement renaming of the current log file to its first rotated version.
    # Check if the file exists before renaming.
    if [[ -f "$FULL_LOG_PATH" ]]; then
        mv "$FULL_LOG_PATH" "$CURRENT_LOG_TO_ONE" || log_message "ERROR" "Failed to rename $FULL_LOG_PATH to $CURRENT_LOG_TO_ONE"
    else
        log_message "WARNING" "Original log file $FULL_LOG_PATH not found after truncation. This might indicate an issue."
    fi

done

# --- Prune Old Archived Logs ---
log_message "INFO" "Pruning archived logs older than $RETENTION_DAYS days."
find "$ARCHIVE_DIR" -type f -name "*.gz" -mtime +$RETENTION_DAYS -delete \
    || log_message "ERROR" "Failed to prune old archived logs. Check permissions."

log_message "INFO" "Log rotation and archiving process finished."
```


### 3.1 Save and Make Executable
1.  Copy the full script content (including your `TODO` solutions) into a file named `rotate_logs.sh` in the same directory as this notebook.
2.  Open your terminal, navigate to that directory, and make the script executable:
    ```bash
    chmod +x rotate_logs.sh
    ```

### 3.2 Run the Script
From your terminal, execute the script:
```bash
./rotate_logs.sh
```

After running, inspect the `app_logs` and `archived_logs` directories. You should see:
-   Original `app.log` and `error.log` truncated or renamed.
-   New `app.log.1` and `error.log.1` files (or higher numbers if you run multiple times).
-   New `.gz` archive files in `archived_logs`.

To fully test the `MAX_ROTATIONS` and `RETENTION_DAYS` features, you'll need to:
1.  Run `generate_dummy_logs()` again (to create fresh logs).
2.  Run `./rotate_logs.sh` multiple times to see the rotation (e.g., app.log.1 -> app.log.2).
3.  Optionally, temporarily modify system time or `mtime` of archived files to test `RETENTION_DAYS`.


## Part 4: Automation and Advanced Challenges (25 points)

Now let's consider how this script would be used in a real system and enhance its capabilities.

### 4.1 Cron Job Setup
Explain, step-by-step, how you would schedule this `rotate_logs.sh` script to run daily at 3:00 AM using `crontab`. Include the specific crontab entry.

**Your Answer:**
*(Write your answer here)*

### 4.2 Error Handling and Notifications
Enhance the script with more robust error handling and notification. Consider:
-   How would you get notified if the script fails (e.g., email, Slack webhook)?
-   What additional checks could you add (e.g., disk space before archiving)?

**Your Answer:**
*(Write your answer here)*

### 4.3 Security Considerations
What are some security best practices to consider when running a log rotation script on a production server? (e.g., permissions, `sudo` usage).

**Your Answer:**
*(Write your answer here)*

### 4.4 Alternative Log Rotation Tools
Mention at least two widely used, dedicated log rotation tools (not just general shell scripting commands) for Unix-like systems. Briefly describe their advantages over a custom shell script.

**Your Answer:**
*(Write your answer here)*